## Preprocessing

In [1869]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [1870]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#application_df = application_df.drop(columns =["EIN","NAME"])

application_df = application_df.drop(columns =["EIN", "NAME"])

application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [1871]:
# number_input_features = len(X_train[0])
# hidden_nodes_layer1 =  100
# hidden_nodes_layer2 = 30
# hidden_nodes_layer3 = 10

# nn = tf.keras.models.Sequential()

# nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
# )

# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# name_counts = application_df['NAME'].value_counts()
# name_counts[name_counts>5]

# names_to_replace = list(name_counts[name_counts <= 5].index)

# for app in names_to_replace:
#     application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# application_df['NAME'].value_counts()

# nn.summary()


In [1872]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [1873]:
# # # Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# print(application_df["APPLICATION_TYPE"].value_counts())

# application_df = application_df.replace(["T25","T14","T29","T15","T17", "T2", "T13", "T12"], "Other")

# application_df

# res  = application_df["APPLICATION_TYPE"].value_counts()

# res

# #change up T replaced values

In [1874]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ["T25","T14","T29","T15","T17", "T2", "T13", "T12", "T9"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [1875]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
app_class = application_df["CLASSIFICATION"].value_counts()

app_class

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [1876]:
# You may find it helpful to look at CLASSIFICATION value counts >1
app_class_2 = app_class[app_class > 10]

app_class_2 = app_class_2.reset_index()
app_class_2

,CLASSIFICATION,count
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
5,C7000,777
6,C1700,287
7,C4000,194
8,C5000,116
9,C1270,114


In [1877]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

app_class_2 = app_class[app_class <= 20]

app_class_2 = app_class_2.reset_index()

classifications_to_replace = list(app_class_2["CLASSIFICATION"])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

#classifications_to_replace

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Other      227
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: count, dtype: int64

In [1878]:
# ask_values = application_df["ASK_AMT"].values

# income_values = application_df["INCOME_AMT"].values

# for i in range (application_df["ASK_AMT"].count()):
#     application_df[i]["Percent of Income"] = ask_values[i]/income_values[i]

# application_df["Percent of Income"] = application_df["ASK_AMT"]/application_df["INCOME_AMT"]


# application_df["Percent of Income"]

# bins = [0,1,9999,24999,99999,499999,999999,4999999,9999999,49999999, 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000]

# bin_labels = ["0","1-9999","10000-24999", "25000-99999", "100000-499999", "500000-999999", "1M-5M", "5M-10M", "10M-50M", "50M+"]

application_df_copy = application_df.copy()

# application_df_copy["ASK_AMT_BINS"] = pd.to_numeric("ASK_AMT_BINS")

# application_df_copy["ASK_AMT_BINS"] = pd.cut(application_df_copy["ASK_AMT"], bins, labels = bin_labels, include_lowest = True)




# application_df_copy.loc[(application_df_copy["ASK_AMT"] > 50000) & (application_df_copy["ASK_AMT"] <= 1000000)]

# application_df_copy.loc[(application_df_copy["ASK_AMT"] < 500000) | (application_df_copy["ASK_AMT"] >= 1000000)]



In [1879]:
print(application_df["APPLICATION_TYPE"].value_counts())

print(application_df["CLASSIFICATION"].value_counts())

print(application_df["INCOME_AMT"].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Other      227
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: count, dtype: int64
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64


In [1880]:
application_df_copy_4 = application_df_copy

application_df_5 = application_df_copy_4.loc[(application_df_copy_4["APPLICATION_TYPE"] != "Other") & (application_df_copy_4["CLASSIFICATION"] != "Other") & (application_df_copy_4["USE_CASE"] != "Other") & (application_df_copy_4["AFFILIATION"] != "Other")]

app = application_df_copy.loc[application_df_copy["INCOME_AMT"] == "0"]

print(application_df_5["APPLICATION_TYPE"].value_counts())


APPLICATION_TYPE
T3     26806
T4      1542
T6      1215
T5      1173
T19     1065
T8       736
T7       724
T10      528
Name: count, dtype: int64


In [1881]:
print(application_df["CLASSIFICATION"].value_counts())

print(app["CLASSIFICATION"].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Other      227
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: count, dtype: int64
CLASSIFICATION
C1000    11681
C2000     4437
C1200     3759
C2100     1544
C3000     1462
C7000      452
C1700      208
Other      155
C5000      106
C4000      105
C1270       82
C2700       67
C7100       60
C2800       58
C1280       46
C1300       39
C1230       35
C2300       28
C1240       23
C1400       22
C7200       19
Name: count, dtype: int64


In [1882]:
print(application_df["INCOME_AMT"].value_counts())

print(app["INCOME_AMT"].value_counts())

# 1    24388
# 0     9911

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64
INCOME_AMT
0    24388
Name: count, dtype: int64


In [1883]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_pd = pd.get_dummies(application_df_5, drop_first = False).astype(int)

dummy_pd

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [1884]:
# # Split our preprocessed data into our features and target arrays
# X = dummy_pd.drop(columns = ['IS_SUCCESSFUL'])
# y = dummy_pd["IS_SUCCESSFUL"]

# # # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78)
print(dummy_pd.columns)

# print(dummy_pd["ASK_AMT_BINS_500000-999999"].value_counts())

print(dummy_pd["INCOME_AMT_0"].value_counts())

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_T10',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7',
       'APPLICATION_TYPE_T8', 'AFFILIATION_CompanySponsored',
       'AFFILIATION_Family/Parent', 'AFFILIATION_Independent',
       'AFFILIATION_National', 'AFFILIATION_Regional', 'CLASSIFICATION_C1000',
       'CLASSIFICATION_C1200', 'CLASSIFICATION_C1230', 'CLASSIFICATION_C1240',
       'CLASSIFICATION_C1270', 'CLASSIFICATION_C1280', 'CLASSIFICATION_C1300',
       'CLASSIFICATION_C1400', 'CLASSIFICATION_C1700', 'CLASSIFICATION_C2000',
       'CLASSIFICATION_C2100', 'CLASSIFICATION_C2300', 'CLASSIFICATION_C2700',
       'CLASSIFICATION_C2800', 'CLASSIFICATION_C3000', 'CLASSIFICATION_C4000',
       'CLASSIFICATION_C5000', 'CLASSIFICATION_C7000', 'CLASSIFICATION_C7100',
       'CLASSIFICATION_C7200', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Pres

In [1885]:
# Split our preprocessed data into our features and target arrays

#  
# X = dummy_pd.drop(columns = ["IS_SUCCESSFUL", "STATUS", "SPECIAL_CONSIDERATIONS_N", "SPECIAL_CONSIDERATIONS_Y", "AFFILIATION_Regional", "AFFILIATION_Other", "CLASSIFICATION_Other", "USE_CASE_Other"])
# y = dummy_pd["IS_SUCCESSFUL"]

# dummy_pd_2= dummy_pd[dummy_pd[["AFFILIATION_Regional", "AFFILIATION_Other", "CLASSIFICATION_Other", "USE_CASE_Other"]].any(axis = 1) != 1]


X = dummy_pd.drop(columns = ["IS_SUCCESSFUL", "STATUS", "SPECIAL_CONSIDERATIONS_Y"])

y = dummy_pd["IS_SUCCESSFUL"]

# # Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

len(X_train.iloc[0])

52

In [1886]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

scale_pd = pd.DataFrame(X_train_scaled)

# pca = PCA(n_components = 0.95)
# X_PCA = pca.fit_transform(X_train_scaled)


## Compile, Train and Evaluate the Model

In [1887]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# #  YOUR CODE GOES HERE



nn = tf.keras.models.Sequential()

x_features = len(X_train.iloc[0])
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=61, input_dim=x_features, activation="relu")
)

# nn.add(
#     tf.keras.layers.Dense(units=14 , activation="relu")
# )

# nn.add(
#     tf.keras.layers.Dense(units=20, activation="relu")
# )

nn.add(
    tf.keras.layers.Dense(units=30, activation="relu")
)
# # # # Second hidden layer

# nn.add(
#     tf.keras.layers.Dense(units=20, activation="relu")
# )


# nn.add(
#     tf.keras.layers.Dense(units=15, activation="relu")
# )

# nn.add(
#     tf.keras.layers.Dense(units=12, activation="relu")
# )


# nn.add(
#     tf.keras.layers.Dense(units=9, activation="relu")
# )

nn.add(
    tf.keras.layers.Dense(units=6, activation="relu")
)

# nn.add(
#     tf.keras.layers.Dense(units=3, activation="relu")
# )

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


c:\Users\librarypc\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [1888]:
# h = kt.HyperParameters()

# activation = h.Choice('activation', ['relu','tanh','sigmoid'])

# nn2 = tf.keras.models.Sequential()

# x_features = len(X_train.iloc[0])
# # First hidden layer
# nn2.add(
#     tf.keras.layers.Dense(units=80, input_dim=x_features, activation=activation)
# )

# nn2.add(
#     tf.keras.layers.Dense(units=65, activation=activation)
# )

# nn2.add(
#     tf.keras.layers.Dense(units=50, activation=activation)
# )

# nn2.add(
#     tf.keras.layers.Dense(units=33, activation=activation)
# )
# # Second hidden layer

# nn2.add(
#     tf.keras.layers.Dense(units=20, activation=activation)
# )


# nn2.add(
#     tf.keras.layers.Dense(units=12, activation=activation)
# )

# nn2.add(
#     tf.keras.layers.Dense(units=6, activation=activation)
# )

# # Output layer
# nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn2.summary()

In [1889]:
#Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [1890]:
# nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=[tf.keras.metrics.Precision()])

In [1891]:
#Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
792/792 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.7003 - loss: 0.5972
Epoch 2/150
792/792 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7362 - loss: 0.5496
Epoch 3/150
792/792 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7316 - loss: 0.5487
Epoch 4/150
792/792 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7329 - loss: 0.5504
Epoch 5/150
792/792 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7338 - loss: 0.5455
Epoch 6/150
 48/792 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7587 - loss: 0.5149

KeyboardInterrupt: 

In [1867]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

264/264 - 1s - 4ms/step - accuracy: 0.7230 - loss: 0.5710
Loss: 0.5709865093231201, Accuracy: 0.7230113744735718


In [1868]:
# Export our model to HDF5 file
# nn.save("nn_HDF5.h5")